In [ ]:
import json
import networkx as nx
import networkit as nk
import numpy as np
from GraphGenerator.preprocessing import dataio
import matplotlib.pyplot as plt
import os
import pickle
import scipy

In [ ]:
graphs = dataio.load_data('exp/SBM_FIRST2/SBM_FIRSTMM_DB_graph_10_2023-Jun-14-21-02-17_12311//FIRSTMM_DB_graph_10_to_sbm.graphs')
#source = dataio.load_data('generated_datasets/src/FIRSTMM_DB/FIRSTMM_DB_2.graph')[0]
#source_nk = nk.nxadapter.nx2nk(source)
source = dataio.load_data('/home/martin/devel/uni/thesis/Generator/GraphGenerator/generated_datasets/src/FIRSTMM_DB/FIRSTMM_DB_graph_10.graph')[0]

In [ ]:
#graphs = pickle.load(open('exp/DD_COLLAB/SBMGNN_DD_graph_1000_2023-Jun-07-08-05-03_348533/DD_graph_1000_to_sbmgnn.graphs', 'rb'))
nx.draw(graphs[0].subgraph(max(nx.connected_components(graphs[0]), key=len)))

In [ ]:
nx.draw(source)

In [ ]:
len(graphs[0].subgraph(max(nx.connected_components(graphs[0]), key=len)).nodes)

In [ ]:
G = nx.Graph()
with open('/home/martin/devel/uni/thesis/benchmark/datasets/FIRSTMMDB/graph_27_0.col') as f:
    for line in f.readlines():
        if line.startswith("e"):      
            s, e = line.split()[1:]
            G.add_edge(int(s)-1, int(e)-1)

In [ ]:
forest = nx.connected_components(G)
nodes = max(forest, key=len)
target = G.subgraph(nodes)


In [ ]:
Filter the graph data

In [ ]:
for file in os.listdir("/home/martin/devel/uni/thesis/benchmark/datasets/gran_DD"):
    if file.endswith("edgelist"):
        path = os.path.join("/home/martin/devel/uni/thesis/benchmark/datasets/gran_DD", file)
        graph = nx.read_edgelist(path)
        if check_connected(graph):
            #print(file.split(".")[0])
            target_path = os.path.join("/home/martin/devel/uni/thesis/Generator/GraphGenerator/generated_datasets/src/DD_clean/", file.split(".")[0]+".graph")
            #nx.write_edgelist(graph, target_path, data=False)
            #pickle.dump([graph], open("{}".format(target_path), "wb"))
            with open(target_path, 'wb') as f:
                pickle.dump([graph], f, protocol=4)

In [ ]:
source

In [ ]:
G = dataio.load_data('generated_datasets/src/DD_clean/_graph_0.graph')

In [ ]:
[G.nodes for G in graphs['31']]

Check if the graphs for that dataset have entries, wich are not connected and if how many.

In [ ]:
graphs = {}
errs = []
for root, dirs, files in os.walk("exp/DCSBM_FIRST"):
    #print(root)
    if "DCSBM_FIRST" in root:
        startid = root.find("graph_")
        endid = root.find("_2023")
        graph_id = root[startid+6:endid]
        graph_files = [file for file in files if ".graphs" in file]
        if len(graph_files)==0:
            errs.append((root, "no graph"))
            continue
        
        
        file_path = os.path.join(root, graph_files[0])
        try:
            g_list = dataio.load_data(file_path)
            
            nnodes = np.array([G.number_of_nodes() for G in g_list]) 
            mask = (nnodes< 10)
            if mask.any():
                errs.append((root, f"{mask.sum()}/{mask.shape[0]} faulty"))
            graphs[graph_id] = graphs.get(graph_id, [])+ [G for i, G in enumerate(g_list) if not mask[i]]
        except (pickle.UnpicklingError, EOFError) as e:
            errs.append((root, e))
def check_connected(x):
    return nx.is_tree(nx.minimum_spanning_tree(x))
#checked_graphs = {}
#for key, value in graphs.items():

#    connected = list(map(check_connected, value))
#    checked_graphs[key] = sum(connected)/len(connected)
#checked_graphs

In [ ]:
graphs

In [ ]:
not_conn = [k for k,v in checked_graphs.items() if v==0.0]
print("Number of Graph sets which have no connected entry", len(not_conn))
print("Ratio of Graph sets which have no connected entry", len(not_conn)/len(checked_graphs.keys()))
total_ratio = [v*5 for k,v in checked_graphs.items()]
print("Total of connected graphs", sum(total_ratio)/(5*len(total_ratio)))

In [ ]:
um(total_ratio)

Some graphs are dissconnected, even though the original graph was not

In [ ]:
only_connected = {}
for key, value in graphs.items():
    connected = list(map(check_connected, value))
    clean_graphs = [graph for i, graph in enumerate(value) if connected[i]]
    if len(clean_graphs) >0:
        only_connected[key] = clean_graphs 

Do spectral analysis for a few graphs

In [ ]:
degrees_source, nNodes_source = get_degree(source)
#nNodes_source = (nNodes_source - np.mean(nNodes_source)) / np.std(nNodes_source)
plt.figure(figsize=(2,4))
plt.plot(degrees_source,nNodes_source)
plt.yscale("log")
plt.title("Node degree")
plt.ylabel("number of nodes")
plt.xlabel("normalized node degree")

In [ ]:
#

def get_degree(graph):
    dd = sorted(nk.centrality.DegreeCentrality(nk.nxadapter.nx2nk(graph), normalized=True).run().scores(), reverse=True)
    return np.unique(dd, return_counts=True)
def get_max_corr(source, gen):
    degrees_source, nNodes_source = get_degree(source)
    degrees_gen, nNodes_gen = get_degree(gen)
    nNodes_gen = (nNodes_gen - np.mean(nNodes_gen)) / (np.std(nNodes_gen)*len(nNodes_gen))
    nNodes_source = (nNodes_source - np.mean(nNodes_source)) / np.std(nNodes_source)
    return scipy.signal.correlate(nNodes_gen, nNodes_source).max()
def plot_corr(source, gen):
    degrees_source, nNodes_source = get_degree(source)
    degrees_gen, nNodes_gen = get_degree(gen)
    #nNodes_gen_norm = np.ones(nNodes_source.shape)
    #corr /= np.max(corr)

    #degrees_gen = (degrees_gen - np.mean(degrees_gen)) / (np.std(degrees_gen)*len(degrees_gen))
    #

    #degrees_gen_norm /= np.linalg.norm(degrees_gen)
    #degrees_source_norm /= np.linalg.norm(degrees_source)

    nNodes_gen_norm = (nNodes_gen - np.mean(nNodes_gen)) / (np.std(nNodes_gen)*len(nNodes_gen))
    nNodes_source_norm = (nNodes_source - np.mean(nNodes_source)) / np.std(nNodes_source)



    corr = scipy.signal.correlate(nNodes_source_norm, nNodes_gen_norm, method='fft')
    lags = scipy.signal.correlation_lags(len(nNodes_source_norm), len(nNodes_gen_norm))
    fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
    ax_orig.plot(degrees_source, nNodes_source)
    ax_orig.set_title('Original signal')
    ax_orig.set_yscale("log")

    ax_noise.plot(degrees_gen, nNodes_gen)
    ax_noise.set_title('Test signal')
    ax_noise.set_yscale("log")
    #plt.xscale("log")
    plt.xlabel("degree")
    #plt.yscale("log")
    plt.ylabel("number of nodes")
    #plt.plot(degrees, numberOfNodes)
    ax_corr.plot(lags, corr)
    plt.show()



In [ ]:
GRAPH = '1058'
graphs = only_connected[GRAPH]#dataio.load_data('exp/SBMGNN_COLLAB/SBMGNN_COLLAB_graph_10_2023-Apr-27-16-23-35_517376/COLLAB_graph_10_to_sbmgnn.graphs')
source = dataio.load_data(f'generated_datasets/src/COLLAB/COLLAB_graph_{GRAPH}.edgelist.graph')[0]
source_nk = nk.nxadapter.nx2nk(source)
correlations = {}    
for gxid in only_connected.keys():
    graphs = only_connected[gxid]#dataio.load_data('exp/SBMGNN_COLLAB/SBMGNN_COLLAB_graph_10_2023-Apr-27-16-23-35_517376/COLLAB_graph_10_to_sbmgnn.graphs')
    source = dataio.load_data(f'generated_datasets/src/COLLAB/COLLAB_graph_{gxid}.edgelist.graph')[0]
    source_nk = nk.nxadapter.nx2nk(source)
    corrs = np.array(list(map(lambda x:get_max_corr(source, x), graphs)))
    correlations[gxid] = corrs
#_id = np.argmax(corrs)
#plot_corr(source, graphs[_id])

In [ ]:
correlations

In [ ]:
import numpy
#G = nk.nxadapter.nx2nk(graphs[0])
def plot_corr_deg(source, gen):
    degrees_source, nNodes_source = get_degree(source)
    degrees_gen, nNodes_gen = get_degree(gen)
    #nNodes_gen_norm = np.ones(nNodes_source.shape)
    #corr /= np.max(corr)

    #degrees_gen = (degrees_gen - np.mean(degrees_gen)) / (np.std(degrees_gen)*len(degrees_gen))
    #

    #degrees_gen_norm /= np.linalg.norm(degrees_gen)
    #degrees_source_norm /= np.linalg.norm(degrees_source)

    nNodes_gen_norm = (nNodes_gen - np.mean(nNodes_gen)) / (np.std(nNodes_gen)*len(nNodes_gen))
    nNodes_source_norm = (nNodes_source - np.mean(nNodes_source)) / np.std(nNodes_source)



    corr = scipy.signal.correlate(nNodes_source_norm, nNodes_gen_norm, method='fft')
    lags = scipy.signal.correlation_lags(len(nNodes_source_norm), len(nNodes_gen_norm))
    fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
    ax_orig.plot(degrees_source, nNodes_source)
    ax_orig.set_title('Original Graph')
    ax_orig.set_yscale("log")
    
    print(degrees_source)
    ax_noise.plot(degrees_gen, nNodes_gen)
    ax_noise.set_title('Generated Graph')
    ax_noise.set_yscale("log")
    #plt.xscale("log")
    plt.xlabel("degree")
    #plt.yscale("log")
    plt.ylabel("number of nodes")
    #plt.plot(degrees, numberOfNodes)
    ax_corr.plot(lags, corr)
    plt.show()
def plot_corr_hist(source, gen):
    hist_source, bins_source = np.histogram(list(nx.eigenvector_centrality(source).values()), bins=10)
    hist_gen, bins_gen = np.histogram(list(nx.eigenvector_centrality(gen).values()), bins=10)
    #nNodes_gen_norm = np.ones(nNodes_source.shape)
    #corr /= np.max(corr)

    #degrees_gen = (degrees_gen - np.mean(degrees_gen)) / (np.std(degrees_gen)*len(degrees_gen))
    #

    #degrees_gen_norm /= np.linalg.norm(degrees_gen)
    #degrees_source_norm /= np.linalg.norm(degrees_source)

    hist_gen_norm = (hist_gen - np.mean(hist_gen)) / (np.std(hist_gen)*len(hist_gen))
    hist_source_norm = (hist_source - np.mean(hist_source)) / np.std(hist_source)



    corr = scipy.signal.correlate(hist_source_norm, hist_gen_norm, method='fft')
    lags = scipy.signal.correlation_lags(len(hist_source_norm), len(hist_gen_norm))
    fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
    ax_orig.plot(bins_source[1:], hist_source)
    ax_orig.set_title('Original Graph')
    ax_orig.set_yscale("log")
    
    print(degrees_source)
    ax_noise.plot(bins_gen[1:], hist_gen)
    ax_noise.set_title('Generated Graph')
    ax_noise.set_yscale("log")
    #plt.xscale("log")
    plt.xlabel("degree")
    #plt.yscale("log")
    plt.ylabel("number of nodes")
    #plt.plot(degrees, numberOfNodes)
    ax_corr.plot(lags, corr)
    plt.show() 
    
def plot_corr_spect(source, gen):
    hist_source, bins_source = np.histogram(list(nx.eigenvector_centrality(source).values()), bins=10)
    hist_gen, bins_gen = np.histogram(list(nx.eigenvector_centrality(gen).values()), bins=10)
    #nNodes_gen_norm = np.ones(nNodes_source.shape)
    #corr /= np.max(corr)

    #degrees_gen = (degrees_gen - np.mean(degrees_gen)) / (np.std(degrees_gen)*len(degrees_gen))
    #

    #degrees_gen_norm /= np.linalg.norm(degrees_gen)
    #degrees_source_norm /= np.linalg.norm(degrees_source)

    hist_gen_norm = (hist_gen - np.mean(hist_gen)) / (np.std(hist_gen)*len(hist_gen))
    hist_source_norm = (hist_source - np.mean(hist_source)) / np.std(hist_source)



    corr = scipy.signal.correlate(hist_source_norm, hist_gen_norm, method='fft')
    lags = scipy.signal.correlation_lags(len(hist_source_norm), len(hist_gen_norm))
    fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, figsize=(4.8, 4.8))
    ax_orig.plot(bins_source[1:], hist_source)
    ax_orig.set_title('Original Graph')
    ax_orig.set_yscale("log")
    
    print(degrees_source)
    ax_noise.plot(bins_gen[1:], hist_gen)
    ax_noise.set_title('Generated Graph')
    ax_noise.set_yscale("log")
    #plt.xscale("log")
    plt.xlabel("degree")
    #plt.yscale("log")
    plt.ylabel("number of nodes")
    #plt.plot(degrees, numberOfNodes)
    ax_corr.plot(lags, corr)
    plt.show()     

GRAPH = '1140'
#graphs = only_connected[GRAPH]#dataio.load_data('exp/SBMGNN_COLLAB/SBMGNN_COLLAB_graph_10_2023-Apr-27-16-23-35_517376/COLLAB_graph_10_to_sbmgnn.graphs')
#source = dataio.load_data(f'generated_datasets/src/ca-condMat/sub_graph_0.edgelist.graph')[0]
#source_nk = nk.nxadapter.nx2nk(source)
#_id = np.argmax(corrs)
largest_cc = graphs[0].subgraph(max(nx.connected_components(graphs[0]), key=len))
plot_corr_deg((source), largest_cc)

In [ ]:
nx.draw(graphs[0].subgraph(largest_cc))

In [ ]:
nx.draw(graphs[0])

In [ ]:
hist, bins = np.histogram(list(nx.eigenvector_centrality(source).values()), bins=10)
plt.plot(bins[1:], hist)

In [ ]:
bins[1:]

In [ ]:
import numpy
#G = nk.nxadapter.nx2nk(graphs[0])
dd = sorted(nk.centrality.DegreeCentrality(nk.nxadapter.nx2nk(graphs[1]), normalized=True).run().scores(), reverse=True)
degrees, numberOfNodes = numpy.unique(dd, return_counts=True)
plt.xscale("log")
plt.xlabel("degree")
plt.yscale("log")
plt.ylabel("number of nodes")
plt.plot(degrees, numberOfNodes)
plt.show()

In [ ]:
nx.eigenvector_centrality(graphs[0], max_iter=1000)

## Compare the validity of the generated solutions
compare the generated graph features to the original sample
Attributes to look for:
 1. The graph is connected
 2. the dregree distribution is similar
 3. Graph edit distance (if aplicable)
 4. 

In [ ]:
#source = dataio.load_data('generated_datasets/src/FIRSTMM_DB/FIRSTMM_DB1.graph')[0]
target = dataio.load_data('exp/E-R/E-R_top1000_2023-Apr-28-11-15-13_19508/top1000_to_e-r.graphs')

In [ ]:
len(target)

In [ ]:
dd = sorted(nx.degree_centrality(source).values(), reverse=True)
degrees, numberOfNodes = numpy.unique(dd, return_counts=True)
plt.xscale("log")
plt.xlabel("degree")
plt.yscale("log")
plt.ylabel("number of nodes")
plt.plot(degrees, numberOfNodes)
plt.show()

In [ ]:
bb = sorted(nx.betweenness_centrality(target).values(), reverse=True)
betweenness, numberOfNodes = numpy.unique(bb, return_counts=True)
plt.xscale("log")
plt.xlabel("betweenness")
plt.yscale("log")
plt.ylabel("number of nodes")
plt.plot(betweenness, numberOfNodes)
plt.show()

In [ ]:
import scipy.signal as signal
signal.freqz(numberOfNodes)

In [ ]:
numberOfNodes

In [ ]:
bb = sorted(nx.simrank_similarity(source).values(), reverse=True)
betweenness, numberOfNodes = numpy.unique(bb, return_counts=True)
plt.xscale("log")
plt.xlabel("betweenness")
plt.yscale("log")
plt.ylabel("number of nodes")
plt.plot(betweenness, numberOfNodes)
plt.show()

In [ ]:
nx.simrank_similarity(nx.cycle_graph(10))